In [ ]:
#https://www.kaggle.com/wayne999/a-simple-genetic-programming/code/code


In [12]:
def mydeap(mungedtrain):
    
    import operator
    import math
    import random
    
    import numpy
    
    from deap import algorithms
    from deap import base
    from deap import creator
    from deap import tools
    from deap import gp
    
    inputs = mungedtrain.iloc[:,2:10].values.tolist()
    outputs = mungedtrain['Survived'].values.tolist()
    
    # Define new functions
    def protectedDiv(left, right):
        try:
            return left / right
        except ZeroDivisionError:
            return 1
    
    pset = gp.PrimitiveSet("MAIN", 8) # eight input
    pset.addPrimitive(operator.add, 2) # a + b
    pset.addPrimitive(operator.sub, 2) # a - b
    pset.addPrimitive(operator.mul, 2) # a * b
    pset.addPrimitive(protectedDiv, 2) # a/b
    pset.addPrimitive(operator.neg, 1) # - a
    pset.addPrimitive(math.cos, 1)
    pset.addPrimitive(math.sin, 1)
    pset.addPrimitive(max, 2)
    pset.addPrimitive(min, 2) # add more?
    pset.addEphemeralConstant("rand101", lambda: random.uniform(-10,10)) # adjust?
    pset.renameArguments(ARG0='x1') #Pclass
    pset.renameArguments(ARG1='x2') #Sex
    pset.renameArguments(ARG2='x3') #Age
    pset.renameArguments(ARG3='x4') #SibSp
    pset.renameArguments(ARG4='x5') #Parch
    pset.renameArguments(ARG5='x6') #Fare
    pset.renameArguments(ARG6='x7') #Cabin
    pset.renameArguments(ARG7='x8') #Embarked

    
    creator.create("FitnessMin", base.Fitness, weights=(1.0,))
    creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)
    
    toolbox = base.Toolbox()
    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3) #
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)
    
    def evalSymbReg(individual):
        # Transform the tree expression in a callable function
        func = toolbox.compile(expr=individual)
        # Evaluate the accuracy
        return sum(round(1.-(1./(1.+numpy.exp(-func(*in_))))) == out for in_, out in zip(inputs, outputs))/len(mungedtrain),
    
    toolbox.register("evaluate", evalSymbReg)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
    
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    
    
    
    
    random.seed(318)
    
    pop = toolbox.population(n=300) #
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.2, ngen=50, stats=mstats,
                                   halloffame=hof, verbose=True) #
    
    print("pset", pset)
    print("master", mstats)
    print(hof[0])
    func2 =toolbox.compile(expr=hof[0])
    return func2
    

In [8]:
import numpy as np
import pandas as pd

In [9]:
def Outputs(data):
    return np.round(1.-(1./(1.+np.exp(-data))))
    
def MungeData(data):
    # Sex
    data.drop(['Ticket', 'Name'], inplace=True, axis=1)
    data.Sex.fillna('0', inplace=True)
    data.loc[data.Sex != 'male', 'Sex'] = 0
    data.loc[data.Sex == 'male', 'Sex'] = 1
    # Cabin
    data.Cabin.fillna('0', inplace=True)
    data.loc[data.Cabin.str[0] == 'A', 'Cabin'] = 1
    data.loc[data.Cabin.str[0] == 'B', 'Cabin'] = 2
    data.loc[data.Cabin.str[0] == 'C', 'Cabin'] = 3
    data.loc[data.Cabin.str[0] == 'D', 'Cabin'] = 4
    data.loc[data.Cabin.str[0] == 'E', 'Cabin'] = 5
    data.loc[data.Cabin.str[0] == 'F', 'Cabin'] = 6
    data.loc[data.Cabin.str[0] == 'G', 'Cabin'] = 7
    data.loc[data.Cabin.str[0] == 'T', 'Cabin'] = 8
    # Embarked
    data.loc[data.Embarked == 'C', 'Embarked'] = 1
    data.loc[data.Embarked == 'Q', 'Embarked'] = 2
    data.loc[data.Embarked == 'S', 'Embarked'] = 3
    data.Embarked.fillna(0, inplace=True)
    data.fillna(-1, inplace=True)
    return data.astype(float)

    

In [11]:
if __name__ == "__main__":
    train = pd.read_csv("./train.csv", dtype={"Age": np.float64}, )
    test = pd.read_csv("./test.csv", dtype={"Age": np.float64}, )
    mungedtrain = MungeData(train)
    print("mungedtrain ", mungedtrain.head(3))
    #GP
    GeneticFunction = mydeap(mungedtrain)
    
    #test
    mytrain = mungedtrain.iloc[:,2:10].values.tolist()
    trainPredictions = Outputs(np.array([GeneticFunction(*x) for x in mytrain]))

    pdtrain = pd.DataFrame({'PassengerId': mungedtrain.PassengerId.astype(int),
                            'Predicted': trainPredictions.astype(int),
                            'Survived': mungedtrain.Survived.astype(int)})
    pdtrain.to_csv('MYgptrain.csv', index=False)
    from sklearn.metrics import accuracy_score
    print(accuracy_score(mungedtrain.Survived.astype(int),trainPredictions.astype(int)))
    
    mungedtest = MungeData(test)
    mytest = mungedtest.iloc[:,1:9].values.tolist()
    testPredictions = Outputs(np.array([GeneticFunction(*x) for x in mytest]))

    pdtest = pd.DataFrame({'PassengerId': mungedtest.PassengerId.astype(int),
                            'Survived': testPredictions.astype(int)})
    pdtest.to_csv('gptest.csv', index=False)

mungedtrain     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Cabin  \
0          1.0       0.0     3.0  1.0  22.0    1.0    0.0   7.2500    0.0   
1          2.0       1.0     1.0  0.0  38.0    1.0    0.0  71.2833    3.0   
2          3.0       1.0     3.0  0.0  26.0    0.0    0.0   7.9250    0.0   

   Embarked  
0       3.0  
1       1.0  
2       3.0  


Exception: Ephemerals with different functions should be named differently, even between psets.